In [1]:
import sys
import warnings
import copy
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, ensemble, metrics, linear_model
# , naive_bayes, metrics, svm, decomposition
import xgboost
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler

warnings.simplefilter('ignore')

input_train_tr = f'{sys.path[0]}/Input/ieee-fraud-detection/train_transaction.csv'
input_test_tr = f'{sys.path[0]}/Input/ieee-fraud-detection/test_transaction.csv'
input_train_id = f'{sys.path[0]}/Input/ieee-fraud-detection/train_identity.csv'
input_test_id = f'{sys.path[0]}/Input/ieee-fraud-detection/test_identity.csv'
output_path = f'{sys.path[0]}/output/sample_submission.csv'

output = pd.DataFrame()

train_transaction = pd.read_csv(input_train_tr)
test_transaction = pd.read_csv(input_test_tr)
train_identity = pd.read_csv(input_train_id)
test_identity = pd.read_csv(input_test_id)

#train_ident = train_transaction['TransactionID'].isin(train_identity['TransactionID'])
#test_ident = test_transaction['TransactionID'].isin(test_identity['TransactionID'])

#train_tr = train_transaction[train_ident == False]
tr_dataset = pd.merge(train_transaction, train_identity,
                      how='left', on='TransactionID')

#test_tr = test_transaction[test_ident == False]
fin_test_id = pd.merge(test_transaction, test_identity,
                       how='left', on='TransactionID')

# label encode the target variable


def encode(dataset):

    encoder = preprocessing.LabelEncoder()

    non_num_cols = [dataset.columns[n] for n, i in enumerate(dataset.dtypes)
                    if i not in ('int64', 'float64')]

    # Filling missing string values with most common value
    for i in non_num_cols:
        dataset[i+'_mod'] = encoder\
            .fit_transform(dataset[i].fillna(dataset[i]
                                             .replace(np.nan, '0', regex=True).mode()[0]))

    col_list = [
        x for x in train_transaction.columns if x not in non_num_cols+['isFraud']]

    #y = dataset['isFraud']

    dataset = dataset[col_list].fillna(0)

    return dataset

# feature engineering


def feature_eng(dataset, y):

    x_train = dataset.fillna(0)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)

    rus = RandomUnderSampler(random_state=42)
    x_train, y = rus.fit_resample(x_train, y)

    return x_train, y

def feature_eng_t(dataset, y):

    x_train = dataset.fillna(0)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    
    return x_train, y

# algorithm testing
def clf(x_train, y_train, x_test, y_test):
    
    variable = np.linspace(0.1, 1.0, 10, endpoint=True)
    
    for n in variable:
        
        RF = ensemble.RandomForestClassifier(n_estimators=32
                                            , n_jobs=-1
                                            , max_depth=8
                                            , min_samples_split = 0.1) #76,71

        #LR = linear_model.LogisticRegression(n_jobs=-1
        #                                    , )#70.48

        RF.fit(x_train, y_train)
        predictions = RF.predict(x_test)

        #LR.fit(x_train, y_train)
        #predictions = LR.predict(x_test)

        accuracy = metrics.accuracy_score(predictions, y_test)
        print(f"variable = {n}, RF:{accuracy}")
    #return predictions

Using TensorFlow backend.


In [2]:
dataset = encode(tr_dataset)

x_train, x_test, y_train, y_test = model_selection.train_test_split(dataset
                                                                   , tr_dataset['isFraud']
                                                                   , test_size = 0.5
                                                                   , random_state=42)

x_train, y_train = feature_eng(x_train, y_train)
x_test, y_test = feature_eng(x_test, y_test)

clf(x_train, y_train, x_test, y_test)

variable = 0.1, RF:0.7666251437332311
variable = 0.2, RF:0.7529704867765428
variable = 0.30000000000000004, RF:0.7510540436949023
variable = 0.4, RF:0.7413760061326179
variable = 0.5, RF:0.7330394787274818
variable = 0.6, RF:0.6667784591797623
variable = 0.7000000000000001, RF:0.5
variable = 0.8, RF:0.5
variable = 0.9, RF:0.5
variable = 1.0, RF:0.5
